In [1]:
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pdb
from IPython.display import display
import ee
import os

In [2]:
import ee
import geemap
service_account = ' boba-account@boba-430314.iam.gserviceaccount.com '
credentials = ee.ServiceAccountCredentials(service_account,'C:/Users/arj26323/OneDrive - University of Georgia/Documents/GEE AUTH/boba-430314-a19be859bfca.json')

ee.Initialize(credentials)

print(geemap.__version__)

0.34.1


In [3]:
Map = geemap.Map(center=[31.539096,-81.422318], zoom=10)

##Adding every plot coordinate
allplots_fc = 'C:/Users/arj26323/Documents/Data/Biomass datasets/Sapelo/GA_allplots_NEW.csv'
fc_all = geemap.csv_to_ee(allplots_fc, latitude = "Latitude", longitude = "Longitude")

In [4]:
##Function to cloud mask from the pixel_qa band of Landsat 5/8 SR data.

#IS THIS SAME BETWEEN SENSORS: https://github.com/giswqs/geemap/discussions/672

def maskL8sr(image):
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)
    # Apply the scaling factors to the appropriate bands.
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    # Replace the original bands with the scaled ones and apply the masks.
    return image.addBands(opticalBands, None, True) \
    .addBands(thermalBands, None, True) \
    .updateMask(qaMask) \
    .updateMask(saturationMask)


In [5]:
#12/08/22 - adding CALIBRATED FLATS
def addFLATSL7(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(1.51 + 12.5*(RED-SWIR)/(RED+SWIR) - 41.2*(NIR-RED)/(NIR+6*RED-7.5*BLUE+1)))', {
            'SWIR': image.select('SR_B5'),
            'NIR': image.select('SR_B4'),
            'RED': image.select('SR_B3'),
            'BLUE': image.select('SR_B1')
        })
    
    return image.addBands(flats.rename('flats'))

def addFLATSL5(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(1.51 + 12.5*(0.972*(RED-SWIR)/(RED+SWIR)-0.008) - 41.2*(0.991*(NIR-RED)/(NIR+6*RED-7.5*BLUE+1)-0.0014)))', {
            'SWIR': image.select('SR_B5'),
            'NIR': image.select('SR_B4'),
            'RED': image.select('SR_B3'),
            'BLUE': image.select('SR_B1')
        })
    
    return image.addBands(flats.rename('flats'))


def addFLATSL8(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(1.51 + 12.5*(0.841*(RED-SWIR)/(RED+SWIR) - 0.019) - 41.2*(0.771*(NIR-RED)/(NIR+6*RED-7.5*BLUE+1) + 0.011)))', {
            'SWIR': image.select('SR_B6'),
            'NIR': image.select('SR_B5'),
            'RED': image.select('SR_B4'),
            'BLUE': image.select('SR_B2')
        })
    
    return image.addBands(flats.rename('flats'))

def addFLATSL9(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(1.51 + 12.5*(1.225*(RED-SWIR)/(RED+SWIR) + 0.096) - 41.2*(1.038* (NIR-RED)/(NIR+6*RED-7.5*BLUE+1) - 0.004)))', {
            'SWIR': image.select('SR_B6'),
            'NIR': image.select('SR_B5'),
            'RED': image.select('SR_B4'),
            'BLUE': image.select('SR_B2')
        })
    
    return image.addBands(flats.rename('flats'))

##MASKING FLATS
def maskFLATS(image):
    mask1 = image.select('flats').lte(0.1) #less than or equal to 0.1 - change?
    return image.updateMask(mask1)

##ADDING NDVI (for min/max variables)
def addL5ndvi(image):
    ndvi = image.expression(
        '(NIR-RED)/(RED+NIR)', {
            'NIR': image.select('SR_B4'),
            'RED': image.select('SR_B3'),
            'GREEN': image.select('SR_B2')
        })
    
    return image.addBands(ndvi.rename('ndvi'))

def addL8ndvi(image):
    ndvi = image.expression(
        '(NIR-RED)/(RED+NIR)', {
            'NIR': image.select('SR_B5'),
            'RED': image.select('SR_B4'),
            'GREEN': image.select('SR_B3')
        })
    
    return image.addBands(ndvi.rename('ndvi'))

In [6]:
##Pixel extraction functions - addDate for dateless images/collections
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('imagedate').toInt())

##For Landsat images:
def rasterExtraction(image):
    feature = image.sampleRegions(
        collection = fc_all,
        scale = 30,
        tileScale = 16 #ADDED 10/6/2022 - make sure it doesn't affect results (see thread below)
    )
    return feature


In [12]:
#Datasets
ls5_collect = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2').filterBounds(fc_all).map(maskL8sr).map(addFLATSL5)
ls7_collect = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2').filterBounds(fc_all).map(maskL8sr).map(addFLATSL7)
ls8_collect = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterBounds(fc_all).map(maskL8sr).map(addFLATSL8)
ls9_collect = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2').filterBounds(fc_all).map(maskL8sr).map(addFLATSL8)

#Merge
ultra_col = ls8_collect.merge(ls5_collect).merge(ls7_collect).merge(ls9_collect).select('flats')

In [36]:
def rasterExtraction(image, fc_subset):
    return image.sampleRegions(
        collection=fc_subset,  # Use the subset of locations
        scale=30,
        tileScale=8  # Adjust tile scale for performance
    ).map(lambda f: f.set('date', image.date().format('YYYY-MM-dd')))

out_dir = os.path.expanduser('~/Documents/Xin data/Landsat')

# Split locations into batches of 100
batch_size = 50
fc_list = fc_all.toList(fc_all.size())  # Convert to list
num_features = fc_all.size().getInfo()

# Calculate the number of batches without math.ceil()
num_batches = (num_features + batch_size - 1) // batch_size  # Equivalent to math.ceil(num_features / batch_size)

years = range(2024, 2025)

for year in years:
    for batch in range(num_batches):
        # Subset locations for this batch
        fc_subset = ee.FeatureCollection(fc_list.slice(batch * batch_size, (batch + 1) * batch_size))

        print(f"Processing {year}, Batch {batch + 1}/{num_batches}...")

        # Filter Landsat images for the year
        subset = ultra_col.filterDate(f"{year}-01-01", f"{year}-12-31")

        # Fix: Use a lambda function to pass `fc_subset` properly
        extracted_features = subset.map(lambda img: rasterExtraction(img, fc_subset)).flatten()

        fc_dict = extracted_features.getInfo()
        features = [feature['properties'] for feature in fc_dict['features']]
        df = pd.DataFrame(features)
        df['date'] = pd.to_datetime(df['date'])

        # Save file in the specified directory
        filename = os.path.join(out_dir, f"landsatflats_{year}_batch{batch + 1}.csv")
        df.to_csv(filename, index=False)
        print(f"Saved {filename}")

print("All batches processed!")


Processing 2024, Batch 1/65...
Saved C:\Users\arj26323/Documents/Xin data/Landsat\landsatflats_2024_batch1.csv
Processing 2024, Batch 2/65...
Saved C:\Users\arj26323/Documents/Xin data/Landsat\landsatflats_2024_batch2.csv
Processing 2024, Batch 3/65...
Saved C:\Users\arj26323/Documents/Xin data/Landsat\landsatflats_2024_batch3.csv
Processing 2024, Batch 4/65...
Saved C:\Users\arj26323/Documents/Xin data/Landsat\landsatflats_2024_batch4.csv
Processing 2024, Batch 5/65...
Saved C:\Users\arj26323/Documents/Xin data/Landsat\landsatflats_2024_batch5.csv
Processing 2024, Batch 6/65...
Saved C:\Users\arj26323/Documents/Xin data/Landsat\landsatflats_2024_batch6.csv
Processing 2024, Batch 7/65...
Saved C:\Users\arj26323/Documents/Xin data/Landsat\landsatflats_2024_batch7.csv
Processing 2024, Batch 8/65...
Saved C:\Users\arj26323/Documents/Xin data/Landsat\landsatflats_2024_batch8.csv
Processing 2024, Batch 9/65...
Saved C:\Users\arj26323/Documents/Xin data/Landsat\landsatflats_2024_batch9.csv
P

In [47]:
#PROPORTION OF TIME FLOODED
#Map locations just to make sure
#Do the same, but with sentinel data
#Inundation intensity
#Rajneesh

# import glob

# path = r'C:/Users/arj26323/Documents/Xin data/Landsat' 
# all_files = glob.glob(path + "/landsatflats_1984_*")

# li = []

# for filename in all_files:
#     df = pd.read_csv(filename, index_col=None, header=0)
#     li.append(df)

# df = pd.concat(li, axis=0, ignore_index=True)

# df

# upd = df[['Latitude', 'Longitude']].drop_duplicates().reset_index()
# upd

# Map.add_points_from_xy(upd, x="Longitude", y="Latitude")
# Map.addLayer(ultra_col.first(),{},'flats')
# Map.addLayer(fc_all,{},'points')
# Map
# fc_all
# Map

In [46]:
import glob

# path = r'C:/Users/arj26323/Documents/Xin data/Landsat' 
# all_files = glob.glob(path + "/*.csv")

# li = []

# for filename in all_files:
#     df = pd.read_csv(filename, index_col=None, header=0)
#     li.append(df)

# df = pd.concat(li, axis=0, ignore_index=True)

# df

,Date,Latitude,Longitude,Plant_Biomass,Plot,Sample_Size,Site,Species_Code,Year,Zone,date,flats
0,10/11/2000,31.538627,-81.422539,516.324,1,23,1,A1,2000,1,1984-04-12,0.003720
1,10/17/2001,31.538627,-81.422539,424.416,1,18,1,A1,2001,1,1984-04-12,0.003720
2,10/16/2002,31.538627,-81.422539,259.784,1,16,1,A1,2002,1,1984-04-12,0.003720
3,10/25/2004,31.538627,-81.422539,436.332,1,18,1,A1,2004,1,1984-04-12,0.003720
4,10/24/2005,31.538627,-81.422539,649.300,1,25,1,A1,2005,1,1984-04-12,0.003720
...,...,...,...,...,...,...,...,...,...,...,...,...
3983823,10/26/2011,31.477098,-81.270523,286.096,2,11,10,A1,2011,2,2024-12-21,0.004613
3983824,10/14/2012,31.477098,-81.270523,371.072,2,12,10,A1,2012,2,2024-12-21,0.004613
3983825,10/13/2013,31.477098,-81.270523,475.344,2,14,10,A1,2013,2,2024-12-21,0.004613
3983826,10/18/2014,31.477098,-81.270523,446.432,2,17,10,A1,2014,2,2024-12-21,0.004613
